# Projet 4

In [20]:
import requests
from bs4 import BeautifulSoup


Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

Exemple `URL_PAGE2` doit retourner :

<pre>{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
  'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
  'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
</pre>

In [21]:
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

def get_prices_from_url(url):
    
    r = requests.get(url)
    content = r.content.decode('utf-8')
    soup = BeautifulSoup(r.content)
    
    dic = {}
    
    for div in soup.find('div', attrs={'class':'pricing-tables pure-g'}).findChildren('div', recursive=False):
        h2 = div.find('h2')
        price = div.find('span', attrs={'class' : 'pricing-table-price'}).text.split()[0]
        storage = div.find('ul', attrs={'class' : 'pricing-table-list'}).find_all('li')[3].text.split()[0]
        database= int(div.find('ul', attrs={'class' : 'pricing-table-list'}).find_all('li')[4].text.split()[0])

        dic[h2.text] = {
            'price': price,
            'storage': storage,
            'databases': database 
        }

    return dic

In [22]:
test = get_prices_from_url('https://kim.fspot.org/cours/page2.html')
print(test)
test2 = get_prices_from_url('https://kim.fspot.org/cours/page3.html')
print(test2)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1}, 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5}, 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1}, 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5}, 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}, 'Privilege': {'price': '$99', 'storage': '1TB', 'databases': 100}}


Ecrire une fonction qui extrait des informations sur une bière de beowulf.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33 

La fonction doit retourner :
<pre>
{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}
</pre>

In [23]:
import json
from multiprocessing import Pool

In [24]:
def extract_beer_infos(url):
    r = requests.get(url)
    content = r.content.decode('utf-8')
    soup = BeautifulSoup(content)

    dic = {}

    name = soup.find('div', {'class' : 'product-detail-info-title'}).find('h1').text 
    note = soup.find('div', {'class' : 'stars'}).attrs['data-percent']
    price = json.loads(soup.find('div', {'class' : 'product pack-product'}).attrs['data-product'])['price']
    volume = int(soup.find('div', {'class' : 'product-subtext'}).text.split('|')[2].split()[0])

    dic= {
        'name': name,
        'note': note,
        'price': price,
        'volume': volume
    }
    return dic

In [25]:
test = extract_beer_infos('https://www.beerwulf.com/fr-fr/p/bieres/hoppebrau-helles.5?scrollTo=relatedProducts')
test

{'name': 'Hoppebrau Helles', 'note': '70', 'price': 44.99, 'volume': 50}

Cette URL retourne un JSON avec une liste de bières :

In [26]:
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"

Ecrire une fonction qui prend l'argument cet URL retourne les informations sur une liste de bière via l'API de beowulf.

Cette fonction doit retourner la liste des informations obtenues par la fonction `extract_beer_infos()` définie ci-dessus.

Chercher comment optimiser cette fonction en utilisant multiprocessing.Pool pour paralléliser les accès web.

Exemple de retour :

<pre>[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
    {'name': 'La Lager Sans Gluten de Vézelay', 'note': 60, 'price': 38.99, 'volume': 25},
    {'name': 'Brasserie De Sutter Brin de Folie', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'La Cristal IPA du Mont Blanc', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'La Parisienne Le Titi Parisien', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
    {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
    {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
    {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]
</pre>

In [27]:

from joblib import parallel_backend

def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    r = requests.get(url)
    data_url = r.json()['items']
    beer_pages = []
       
    for index , value in enumerate(data_url):
        beer_pages.append('https://www.beerwulf.com' + value['contentReference'])

    # Sequential version (slow):
    beers = []
    for url2 in beer_pages:
        beers.append(extract_beer_infos(url2))
    
#     # Parallel version (faster):
    
#     beers = []
#     with Pool(len(beer_pages)) as p:
#         beers = p.map(extract_beer_infos, beer_pages)
    
    
    return beers

In [ ]:
beers = extract_beer_list_infos(URL_BEERLIST_FRANCE)
beers

In [ ]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        # We should have 11 French beers:
        self.assertIsInstance(infos, list)
        self.assertEqual(len(infos), 11)
        # All of them are 25cl or 33cl:
        for beer in infos:
            self.assertIn(beer['volume'], [25, 33])

            
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [ ]:
if __name__ == '__main__':
    run_tests()